In [135]:
import pandas as pd

In [136]:
data_proc = pd.read_csv("medprocner_tsv_train_subtask1.tsv", delimiter="\t")
data_proc

,filename,ann_id,label,start_span,end_span,text
0,es-S0210-56912007000900007-3,T1,PROCEDIMIENTO,609,643,tomografía computarizada abdominal
1,es-S0210-56912007000900007-3,T2,PROCEDIMIENTO,799,824,intubada orotraquealmente
2,es-S0210-56912007000900007-3,T3,PROCEDIMIENTO,827,861,conectada a un respirador mecánico
3,es-S0210-56912007000900007-3,T4,PROCEDIMIENTO,879,908,tratamiento a base de fluidos
4,es-S0210-56912007000900007-3,T5,PROCEDIMIENTO,1362,1374,colonoscopia
...,...,...,...,...,...,...
11060,es-S1134-80462009000800005-1,T7,PROCEDIMIENTO,1768,1770,EA
11061,es-S1134-80462009000800005-1,T8,PROCEDIMIENTO,2033,2035,EA
11062,es-S1134-80462009000800005-1,T9,PROCEDIMIENTO,3113,3115,EA
11063,es-S1134-80462009000800005-1,T10,PROCEDIMIENTO,1420,1422,EA


In [155]:
f_out = open("MedProcNER_training.json", 'w', encoding="UTF-8")
f_out.write('{"version": "0.1.0",\n"data": [')
test_path = "txt"
test_files = os.listdir(test_path)
for f in test_files:
    with open(test_path+'/'+f, "r", encoding="UTF-8") as ftest:
        texto = ftest.read()
    fl = data_proc[data_proc['filename'] == f[:-4]].sort_values('start_span', ascending=True)
    procedures = []
    for i in range(len(fl)):
        procedures.append(fl['text'].iloc[i])
    for p in procedures:        
        tagged = [a+"___I#-#P#R#O#C#E#D#U#R#E" for a in p.split()]
        tagged[0] = tagged[0].replace('I#-#P#R#O#C#E#D#U#R#E', 'B#-#P#R#O#C#E#D#U#R#E')
        txt_tag = ' '.join( tagged )
        #texto = re.sub(texto[start:end],txt_tag,texto)
        texto = texto.replace(p,txt_tag)
        texto = texto.replace('. ',' . ')
        texto = texto.replace('.\n',' . ')
        texto = texto.replace(', ',' , ')
        texto = texto.replace(': ',' : ')
        texto = texto.replace('! ',' ! ')
        texto = texto.replace('? ',' ? ')
        texto = texto.replace('; ',' ; ')
        texto = texto.replace(') ',' ) ')
        texto = texto.replace('" ',' " ')
        texto = texto.replace(' (',' ( ')
        texto = texto.replace(' "',' " ')
        texto = texto.replace('  ', ' ')
    texto = texto.split()
    ner_tags = []
    tokens = []
    for w in texto:
        if "B#-#P#R#O#C#E#D#U#R#E" in w:
            wp = w.replace("___B#-#P#R#O#C#E#D#U#R#E", "")
            wp = wp.replace("___I#-#P#R#O#C#E#D#U#R#E", "") #To clean multiple tags
            tokens.append(wp)
            ner_tags.append("B-PROCEDURE")
        elif "I#-#P#R#O#C#E#D#U#R#E" in w:
            wp = w.replace("___I#-#P#R#O#C#E#D#U#R#E", "")
            tokens.append(wp)
            ner_tags.append("I-PROCEDURE")
        else:
            tokens.append(w)
            ner_tags.append("O")
        if w == ".":
            dicc = {"ner_tags": ner_tags, "tokens": tokens}
            f_out.write(str(dicc)+',\n')
            ner_tags = []
            tokens = []
    #print(tokens, len(tokens))
    #print(ner_tags, len(ner_tags))
f_out.write("]}")
f_out.close()